# SurenaIV Offline Walk
* CoM Pattern Generation using DCM 
* 5th order ankle Trajectory
* Geometric IK

Start Simulator and Spawn Robot

In [5]:
import pybullet
import time
import pybullet_data
import numpy as np
import Robot
import Ankle
import DCM

In [ ]:
phisycsClient = pybullet.connect(pybullet.GUI)
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath())

In [ ]:
pybullet.resetSimulation()
planeID = pybullet.loadURDF("plane.urdf")
pybullet.setGravity(0,0,-9.81)
robotID = pybullet.loadURDF("surena4_1.urdf",
                         [0.0,0.0,0.0],useFixedBase = 1)
pybullet.setRealTimeSimulation(1)

Generate Trajectories

Spin Offline...